In [1]:
import sqlite3 as lite
from pathlib import Path
import pandas as pd

hedging_path = Path("/private/var/mobile/Containers/Shared/AppGroup/7B4E60FC-E291-4D70-A49B-D500197061AB/File Provider Storage/Repositories/hedging")

bt_path = hedging_path / 'base_de_trabajo.sqlite'

print(bt_path.resolve())


/private/var/mobile/Containers/Shared/AppGroup/7B4E60FC-E291-4D70-A49B-D500197061AB/File Provider Storage/Repositories/hedging/base_de_trabajo.sqlite


In [27]:
bt_conn = lite.connect(str(bt_path))

fondo = 'BBVAPC'

sql = f"""
SELECT instrumento,
       curr_code, 
       mtm, 
       num_titulos, 
       mtm/num_titulos as precio 
FROM grisk
WHERE fondo = '{fondo}'
  AND (instrumento LIKE '56%' OR 
       instrumento LIKE 'FD%' OR 
       instrumento LIKE 'CH%');
"""

df = pd.read_sql(sql, bt_conn, index_col='instrumento')

usd = df.loc['CH_BACOMER_1602926', 'precio']
eur = df.loc['CHE_BBVAES_1538071', 'precio']

fx = pd.DataFrame(data = {'curr_code':['USD', 'EUR'], 'fx':[usd,eur]})

instrumentos = ",".join("'" + i + "'" for i in df.index.to_list())

sql = f"""
SELECT *
FROM pnl
WHERE instrumento IN ({instrumentos});
"""

pnl = pd.read_sql(sql, bt_conn, index_col='instrumento')
pnl = pnl[pnl.columns[1:]]


sql = f"""
SELECT instrumento, precio
FROM precios
WHERE instrumento IN ({instrumentos});
"""

precios = pd.read_sql(sql, bt_conn, index_col='instrumento')

pnl = pnl.add(precios.precio, axis=0)

simulacion = pnl = precios.merge(pnl, left_index=True, right_index=True)

df = df[df.columns[:-1]]
df = df.loc[df.index.str.contains("FD_|56SP_")]

bt_conn.close()

In [3]:
monto_de_cobertura = df.loc[df.index.str.contains("FD_"),"num_titulos"].sum()*10000

In [4]:
etfs = df.loc[df.index.str.contains("56SP_")].copy(deep=True)
etfs = etfs.merge(fx, left_on='curr_code', right_on='curr_code')

etfs["mtm_usd"] = etfs.mtm/etfs.fx

monto_a_cubrir = etfs.mtm_usd.sum()


In [54]:
monto_a_cubrir

14028950.667122759

In [55]:
monto_de_cobertura

-14310000.0

In [29]:
df

,curr_code,mtm,num_titulos
instrumento,,,
56SP_BRFI2UH_LX,USD,1.385359e+08,64380.0
56SP_BTAM2AU_LX,EUR,1.365039e+08,6581.0
FD_DEUA_MR23,MXN,-1.534540e+05,-194.0
FD_DEUA_JN23,MXN,-1.010629e+06,-1237.0


In [8]:
pnl = precios.merge(pnl, left_index=True, right_index=True)

In [33]:
simulacion.loc['56SP_BTAM2AU_LX'] / simulacion.loc['CHE_BBVAES_1538071']

precio    1054.169994
E1        1054.255186
E2        1054.828539
E3        1055.110178
E4        1057.088778
             ...     
E496      1056.685145
E497      1069.562005
E498      1032.207016
E499      1046.583700
E500      1049.210194
Length: 501, dtype: float64

In [34]:
simulacion

,precio,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,E491,E492,E493,E494,E495,E496,E497,E498,E499,E500
instrumento,,,,,,,,,,,,,,,,,,,,,
CHE_BBVAES_1538071,19.676256,19.663775,19.566481,19.615891,19.697501,19.636801,19.677525,19.757503,19.754009,19.778210,...,19.801585,19.987417,19.349267,19.612936,19.543051,19.565968,19.356813,19.960346,19.737889,19.853147
CH_BBVAES_2646035,19.535600,19.524031,19.407787,19.501130,19.582289,19.430799,19.484381,19.618745,19.541444,19.626158,...,19.495212,19.608291,19.434601,19.488380,19.535797,19.446749,19.466751,19.404771,19.465841,19.622182
56SP_BTAM2AU_LX,20742.118660,20730.637190,20639.282525,20696.926330,20822.007736,20630.845621,20691.525880,20860.922136,20748.323766,20873.373272,...,20685.402936,20788.707595,20609.771930,20665.012958,20779.990400,20675.068001,20703.311213,20603.209466,20657.352772,20830.124317
56SP_BRFI2UH_LX,2151.846340,2151.345014,2136.999665,2149.207581,2158.732584,2140.878735,2145.626904,2160.616905,2152.490075,2163.373450,...,2160.651952,2164.203847,2148.741166,2143.909181,2152.456489,2142.059444,2143.676393,2142.306155,2143.382416,2165.905897
CH_BACOMER_1602926,19.535600,19.524031,19.407787,19.501130,19.582289,19.430799,19.484381,19.618745,19.541444,19.626158,...,19.495212,19.608291,19.434601,19.488380,19.535797,19.446749,19.466751,19.404771,19.465841,19.622182
FD_DEUA_JN23,945.177399,789.133243,-364.087301,570.675739,1386.952962,-246.930339,410.204300,1746.436818,1049.542871,1849.246177,...,530.450374,1842.561746,-161.656723,288.987282,910.367659,-70.997222,321.360902,-352.206309,123.434898,1835.540863
FD_DEUA_MR23,817.980675,710.877271,-460.668260,464.029330,1273.288175,-313.897292,262.701558,1585.484507,929.206784,1720.571821,...,343.578586,1619.035925,-217.909362,297.328237,806.674418,-135.591654,148.945538,-568.917136,74.420040,1689.995983
